# 1. RAG chain 구현 구문

In [2]:
import os
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate, ChatMessagePromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableWithMessageHistory
from langchain_community.tools import TavilySearchResults
from langchain_core.documents import Document

from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

from ragas import EvaluationDataset, RunConfig, evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper


from textwrap import dedent
from operator import itemgetter

from dotenv import load_dotenv
load_dotenv()

True

In [3]:
COLLECTION_NAME = os.getenv("COLLECTION_NAME")
PERSIST_DIRECTORY = os.getenv("PERSIST_DIRECTORY")
EMBEDDING_MODEL_NAME = os.getenv("EMBEDDING_NAME")
embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)
MODEL_NAME = os.getenv("MODEL_NAME")
print(COLLECTION_NAME)
print(PERSIST_DIRECTORY)
print(EMBEDDING_MODEL_NAME)
print(MODEL_NAME)

bluer_db_openai
vector_store/chroma/bluer_db
text-embedding-3-small
gpt-4o-mini


In [4]:
########################################################
# vector_db에서 데이터 불러오기
########################################################

vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)


# GPT Model 생성
model = ChatOpenAI(
    model=MODEL_NAME,
    temperature=0 
)


retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 50,
        "fetch_k": 200,
        "lambda_mult": 0.5,
        # "filters": {"리본개수": {"$gte": 0}}
    }
)


prompt_template = ChatPromptTemplate.from_messages([
    ("system", dedent("""
        당신은 한국의 식당을 소개하는 인공지능 비서입니다. 
        반드시 질문에 대해서 [context]에 주어진 내용을 바탕으로 답변을 해주세요. 
        질문에 '리본개수', '평점', '몇 개'라는 키워드가 포함된 경우, [context]에서 "리본개수" 항목을 확인해 답변하세요.
        리본개수는 평점과 같은 의미를 가집니다.
        [context]
        {context}
    """)),
    ("human", "{question}")
])


#########################################
# Chain 생성
#########################################

def content_from_doc(docs:list[Document]):
    return "\n\n".join([d.page_content for d in docs])


chain =  {'context': retriever  | RunnableLambda(content_from_doc), 'question': RunnablePassthrough()}  | prompt_template | model | StrOutputParser()

In [5]:
QUERY = "서울 서대문구 근처 음식점 중 리본개수가 2개인 맛집을 추천해줘."
response = chain.invoke(QUERY)
print(response)

서울 서대문구 근처에서 리본개수가 2개인 맛집은 '내일도두부'입니다. 이곳은 두부 전문점으로, 두부전골, 버섯불고기두부밥, 얼큰순두부 등의 메뉴를 제공합니다. 가격대는 1~2만원대이며, 영업시간은 10:00~21:30입니다.


In [51]:
QUERY = "리본개수가 1개 이상인 식당만 알려주세요."
response = chain.invoke(QUERY)
print(response)

리본개수가 1개 이상인 식당은 다음과 같습니다:

1. 버섯찌개경주집 - 리본개수: 1
2. 이조식탁 - 리본개수: 1
3. 진진만두 - 리본개수: 1
4. 르비엣 - 리본개수: 1
5. 부민옥 - 리본개수: 1


In [3]:
QUERY = "리본개수가 2개인 식당을 추천해주세요"
response = chain.invoke(QUERY)
print(response)

리본개수가 2개인 식당으로는 다음과 같은 곳들이 있습니다:

1. 팔레드신: 중식, 모던차이니즈, 홍콩식중식, 북경오리를 전문으로 하는 식당입니다. 서울특별시 중구 퇴계로에 위치하고 있으며, 화려한 인테리어와 수준 높은 북경오리와 광동요리를 제공합니다.

2. 핑하오: 일반중식을 제공하는 식당으로, 서울특별시 마포구 마포대로에 위치하고 있습니다. 신라호텔, 리츠칼튼호텔 출신의 왕병호 셰프가 운영하며, 합리적인 가격으로 수준급의 중식 요리를 즐길 수 있습니다.

3. 백리향: 일반중식을 제공하는 식당으로, 서울특별시 영등포구 63로에 위치하고 있습니다. 정통 스타일의 중식을 즐길 수 있으며, 비즈니스나 상견례 장소로도 인기가 높습니다.

4. 삼청각한식당: 한식(일반한식), 한정식을 제공하는 식당으로, 서울특별시 성북구 대사관로에 위치하고 있습니다. 정갈한 궁중 음식과 함께 전통문화와 예술을 경험할 수 있는 곳입니다.

5. BLT스테이크: 기타, 스테이크를 제공하는 식당으로, 서울특별시 종로구 청계천로에 위치하고 있습니다. 뉴욕의 3대 스테이크로 불리는 곳으로, 뉴욕채끝등심과 한우투뿔안심이 대표 메뉴입니다.


In [ ]:
QUERY = "예약가능한 한식당 추천해줘."
response = chain.invoke(QUERY)
print(response)

다음은 예약 가능한 한식당 목록입니다:

1. 아리랑가든
   - 음식종류: 한식(일반한식), 버섯전골, 일반한식
   - 전화번호: 043-744-0203
   - 도로명주소: 충청북도 영동군 영동읍 학산영동로 1041

2. 연춘식당
   - 음식종류: 한식(민물어패류), 장어, 한식(가금류), 닭구이
   - 전화번호: 041-545-2866
   - 도로명주소: 충청남도 아산시 신정호길 67

3. 명동식당
   - 음식종류: 한식(일반한식), 한정식
   - 전화번호: 061-433-2147
   - 도로명주소: 전라남도 강진군 강진읍 서성안길 5

4. 삼청각한식당
   - 음식종류: 한식(일반한식), 한정식
   - 전화번호: 02-765-3000
   - 도로명주소: 서울특별시 성북구 대사관로 3

5. 애성회관
   - 음식종류: 한식(탕/국/찌개/전골), 곰탕, 한식(육류), 수육
   - 전화번호: 02-352-0303
   - 도로명주소: 서울특별시 중구 남대문로5길 23

6. 대한식당
   - 음식종류: 한식(육류), 소불고기
   - 전화번호: 061-763-0095
   - 도로명주소: 전라남도 광양시 광양읍 매일시장길 12-15

7. 경희식당
   - 음식종류: 한식(일반한식), 한정식
   - 전화번호: 043-543-3736
   - 도로명주소: 충청북도 보은군 속리산면 사내7길 11-4

8. 삼호식당
   - 음식종류: 한식(일반한식), 일반한식, 산채비빔밥
   - 전화번호: 041-836-5038
   - 도로명주소: 충청남도 부여군 외산면 무량로 190

9. 은진손칼국수
   - 음식종류: 한식(면류), 칼국수
   - 전화번호: 041-741-0612
   - 도로명주소: 충청남도 논산시 은진면 매죽헌로25번길 8

10. 역삼동북어집
    - 음식종류: 한식(어패류), 북어찜
    - 전화번호: 02-558-6605
    - 도로명주소: 서울특별시 강남구 논현로85길 5-14

이 외에도 다양한 한식당이 

In [ ]:
QUERY = "리본개수가 2개인 식당을 몇 곳 추천해주세요"
response = chain.invoke(QUERY)
print(response)

리본개수가 2개인 식당은 다음과 같습니다:

1. 백리향
2. 팔레드신
3. 핑하오
4. 성민양꼬치
5. BLT스테이크

이 식당들은 각각 다양한 종류의 요리를 제공하며, 높은 평점을 받은 곳들입니다.


In [49]:
QUERY = "리본개수가 2개인 식당을 다섯 곳을 추천해주세요"
response = chain.invoke(QUERY)
print(response)

다음은 리본개수가 2개인 식당 다섯 곳입니다:

1. 백리향
2. 라씨에트
3. 핑하오
4. 성민양꼬치
5. 팔레드신


In [5]:
QUERY = "식당을 추천해주세요"
response = chain.invoke(QUERY)
print(response)

추천할 만한 식당은 여러 곳이 있습니다. 몇 가지를 소개해드리겠습니다.

1. **도취**
   - 음식종류: 바, 다이닝바, 와인바
   - 전화번호: 02-723-1288
   - 메뉴: 육회타르타르, 크림치즈곶감말이, 마라순두부전골 등
   - 영업시간: 17:00~24:00
   - 주소: 서울특별시 종로구 옥인길 23-6
   - 특징: 아지트 분위기의 한식 다이닝 와인바로, 육회타르타르가 시그니처 메뉴입니다.

2. **삼청각한식당**
   - 음식종류: 한식(일반한식), 한정식
   - 전화번호: 02-765-3000
   - 메뉴: 다양한 한정식 코스
   - 영업시간: 12:00~15:00/18:00~21:00
   - 주소: 서울특별시 성북구 대사관로 3
   - 특징: 전통문화시설에서 정갈한 궁중 음식을 경험할 수 있는 곳입니다.

3. **홍운장**
   - 음식종류: 중식, 일반중식
   - 전화번호: 02-558-3666
   - 메뉴: 군만두, 짬뽕, 탕수육 등
   - 영업시간: 11:30~21:00
   - 주소: 서울특별시 강남구 삼성로 341
   - 특징: 오랜 역사를 자랑하는 중국집으로, 짬뽕과 짜장면이 추천 메뉴입니다.

이 외에도 다양한 식당이 있으니, 원하는 음식 종류나 분위기에 따라 선택하시면 좋을 것 같습니다!


In [25]:
QUERY = "프랑스식을 추천해주세요"
response = chain.invoke(QUERY)
print(response)

다음은 프랑스식 음식을 제공하는 식당들입니다:

1. **파사주**
   - 전화번호: 02-332-8994
   - 가격대: 2~5만원대
   - 메뉴: 잠봉샐러드, 프렌치어니언수프, 파흐망티에 등
   - 영업시간: 11:30~15:00 / 17:00~22:00 (주말 11:00~15:00 / 16:30~22:00)
   - 주소: 서울특별시 마포구 백범로20길 24-5
   - 예약 가능

2. **라브리**
   - 전화번호: 02-739-8930
   - 가격대: 5~10만원대
   - 메뉴: 양갈비스테이크, 런치세트, 저녁세트 등
   - 영업시간: 11:30~15:00 / 17:30~21:30 (토요일 12:00~15:00 / 17:30~21:30)
   - 주소: 서울특별시 종로구 종로 1
   - 예약 불가능

3. **르셰프블루**
   - 전화번호: 02-6010-8088
   - 가격대: 10~15만원대
   - 메뉴: 런치코스, 디너코스 등
   - 영업시간: 11:30~15:00 / 17:30~22:00
   - 주소: 서울특별시 중구 청파로 435-10
   - 예약 가능

4. **빠리가옥**
   - 전화번호: 02-6083-1626
   - 가격대: 2~5만원대
   - 메뉴: 에스카르고, 비프부르기뇽, 살치살스테이크 등
   - 영업시간: 11:30~15:00 / 17:00~22:00 (주말 11:30~22:00)
   - 주소: 서울특별시 종로구 수표로28길 33-4
   - 예약 가능

이 외에도 다양한 프랑스식 식당이 있으니 참고하시기 바랍니다!


In [77]:
QUERY = "가족과 함께 갈만한 식당을 추천해주세요"
response = chain.invoke(QUERY)
print(response)

가족과 함께 가기 좋은 식당으로는 '고루'를 추천드립니다. 이 식당은 인천광역시 서구에 위치한 전통 한정식집으로, 멋들어진 한옥 외관과 운치 있는 분위기가 돋보입니다. 음식 맛이 자극적이지 않고 메뉴가 다채롭게 구성되어 있어 가족 모임에 적합합니다. 방이 개별로 나뉘어 있어 프라이빗한 식사가 가능하며, 상견례나 모임을 하기에 좋은 장소입니다.
